<a href="https://colab.research.google.com/github/satishgaurav/Molecule-Classification/blob/master/Copy_of_chemprop_colab_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Mar 10 19:50:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [3]:
!pip install chemprop
!pip install rdkit-pypi  # should be included in above after Chemprop v1.6 release

import chemprop
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.decomposition import PCA

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 40.6 MB/s eta 0:00:00
  Created wheel for typed-argument-parser: filename=typed_argument_parser-1.9.0-py3-none-any.whl size=25615 sha256=1614bc6236c967b98cc64f3a0c856c26c733cafb21f341abbd0e9b95d0297e98
  Stored in directory: /root/.cache/pip/wheels/f0/94/0f/9539f578bed7e1bd423c702e403712f5ee8989f831a71db000
Successfully built typed-argument-parser
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 22.2 MB/s eta 0:00:00


In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
hiv_df = pd.read_csv("HIV.csv")
hiv_df.head()

,smiles,activity,HIV_active
0,CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)...,CI,0
1,C(=Cc1ccccc1)C1=[O+][Cu-3]2([O+]=C(C=Cc3ccccc3...,CI,0
2,CC(=O)N1c2ccccc2Sc2c1ccc1ccccc21,CI,0
3,Nc1ccc(C=Cc2ccc(N)cc2S(=O)(=O)O)c(S(=O)(=O)O)c1,CI,0
4,O=S(=O)(O)CCS(=O)(=O)O,CI,0


In [6]:
hiv_df.describe()

,HIV_active
count,41127.000000
mean,0.035086
std,0.184001
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [7]:
unique_values = hiv_df['HIV_active'].unique()
print(f"Unique values in 'HIV_active': {unique_values}")

Unique values in 'HIV_active': [0 1]


In [8]:
unique_values = hiv_df['smiles'].unique()
print(f"Unique values in 'smiles': {unique_values}")
print(f"length of uniqe value: {len(unique_values)}")

Unique values in 'smiles': ['CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)=[O+]2'
 'C(=Cc1ccccc1)C1=[O+][Cu-3]2([O+]=C(C=Cc3ccccc3)CC(c3ccccc3)=[O+]2)[O+]=C(c2ccccc2)C1'
 'CC(=O)N1c2ccccc2Sc2c1ccc1ccccc21' ...
 'Cc1ccc(N2C(=O)C3c4[nH]c5ccccc5c4C4CCC(C(C)(C)C)CC4C3C2=O)cc1'
 'Cc1cccc(N2C(=O)C3c4[nH]c5ccccc5c4C4CCC(C(C)(C)C)CC4C3C2=O)c1'
 'CCCCCC=C(c1cc(Cl)c(OC)c(-c2nc(C)no2)c1)c1cc(Cl)c(OC)c(-c2nc(C)no2)c1']
length of uniqe value: 41127


In [9]:
# Filter rows where 'your_column' is not equal to 1 or 0
filtered_df = hiv_df[(hiv_df['HIV_active'] != 1) & (hiv_df['HIV_active'] != 0)]
filtered_df

,smiles,activity,HIV_active


In [10]:
# Filter rows where 'target_column' is equal to 1h
hiv_df_filtered_active = hiv_df[hiv_df['HIV_active'] == 1]
hiv_df_filtered_active

,smiles,activity,HIV_active
11,O=C(O)Cc1ccc(SSc2ccc(CC(=O)O)cc2)cc1,CM,1
16,NNP(=S)(NN)c1ccccc1,CM,1
80,O=Nc1ccc(O)c(N=O)c1O,CM,1
203,Oc1ccc(Cl)cc1C(c1cc(Cl)ccc1O)C(Cl)(Cl)Cl,CM,1
234,NNC(=O)c1ccccc1SSc1ccccc1C(=O)NN,CM,1
...,...,...,...
41090,Cc1cn(COCCCOCC(=O)c2ccccc2)c(=O)[nH]c1=O,CM,1
41092,Cc1cn(C2CC3C(COC(CCC[Se]c4ccccc4)N3O)O2)c(=O)[...,CM,1
41093,Cc1cn(C2CC3C(COC(CCCC[Se]c4ccccc4)N3O)O2)c(=O)...,CM,1
41098,Cc1cn(C2CC3C(COC(CC[Se]C#N)N3O)O2)c(=O)[nH]c1=O,CM,1


In [11]:
# Filter rows where 'target_column' is equal to 1h
hiv_df_filtered_inactive = hiv_df[hiv_df['HIV_active'] == 0]
hiv_df_filtered_inactive = hiv_df_filtered_inactive.sample(n=1500, axis=0, random_state=42)
hiv_df_filtered_inactive

,smiles,activity,HIV_active
2428,O=C1c2ccccc2-c2nc3ccccc3nc21,CI,0
6197,O=C(CSc1cc(-c2ccc(Cl)cc2)s[s+]1)c1ccccc1,CI,0
17138,O=C(C=Nc1ccccc1C(=O)O)c1ccco1,CI,0
12261,CCCCCCCCCCCCCCCCCC[N+](C)(C)Cc1ccc(C[N+](C)(C)...,CI,0
3588,N#CSC1CCCCCCC1SC#N,CI,0
...,...,...,...
18477,CC(=O)OC1(C#N)CC2OC1C1C2N1C(=O)OC(C)(C)C,CI,0
1189,CCOC(=O)C1Cc2cc(C)c(C)cc2N(C)C1=O,CI,0
36657,CCOC(=O)N1CCN(c2ccc3c(C)cc(C)nc3n2)CC1,CI,0
27919,CN(C)C=Nc1ccc2c3c(cccc13)-c1ccccc1-2,CI,0


In [13]:
hiv_df_sampled = pd.concat([hiv_df_filtered_active, hiv_df_filtered_inactive], axis=0, ignore_index=True)
hiv_df_sampled

,smiles,activity,HIV_active
0,O=C(O)Cc1ccc(SSc2ccc(CC(=O)O)cc2)cc1,CM,1
1,NNP(=S)(NN)c1ccccc1,CM,1
2,O=Nc1ccc(O)c(N=O)c1O,CM,1
3,Oc1ccc(Cl)cc1C(c1cc(Cl)ccc1O)C(Cl)(Cl)Cl,CM,1
4,NNC(=O)c1ccccc1SSc1ccccc1C(=O)NN,CM,1
...,...,...,...
2938,CC(=O)OC1(C#N)CC2OC1C1C2N1C(=O)OC(C)(C)C,CI,0
2939,CCOC(=O)C1Cc2cc(C)c(C)cc2N(C)C1=O,CI,0
2940,CCOC(=O)N1CCN(c2ccc3c(C)cc(C)nc3n2)CC1,CI,0
2941,CN(C)C=Nc1ccc2c3c(cccc13)-c1ccccc1-2,CI,0


In [16]:
# Randomly shuffle rows
hiv_df_sampled = hiv_df_sampled.sample(frac=1, random_state=42)

In [17]:
hiv_df_sampled.head()

,smiles,activity,HIV_active
240,Cc1cc2c(c(=O)o1)C1=S(SC(c3ccccc3)=C1)S2,CM,1
2325,N#CN1CCC=C(c2cc3ccccc3[nH]2)C1,CI,0
1676,CCC1SC(C)C(=O)NC1=O,CI,0
1952,O=C1CC2(CCN(Cc3ccccc3)CC2)CC(=O)N1,CI,0
677,CC(=O)OC1SC(c2c(F)cccc2F)n2c1nc1ccccc12,CM,1


In [18]:
hiv_df_sampled.to_csv('HIV_2.csv', index=False)
# .drop(['activity'], axis=1).
hiv_df_sampled_2 = pd.read_csv("HIV_2.csv")
hiv_df_sampled_2.head()
hiv_df_sampled_2.tail()

,smiles,activity,HIV_active
0,Cc1cc2c(c(=O)o1)C1=S(SC(c3ccccc3)=C1)S2,CM,1
1,N#CN1CCC=C(c2cc3ccccc3[nH]2)C1,CI,0
2,CCC1SC(C)C(=O)NC1=O,CI,0
3,O=C1CC2(CCN(Cc3ccccc3)CC2)CC(=O)N1,CI,0
4,CC(=O)OC1SC(c2c(F)cccc2F)n2c1nc1ccccc12,CM,1


,smiles,activity,HIV_active
2938,O=C(CS)Nc1cccc(O)c1,CI,0
2939,O=C(Nc1ccc(N=Nc2ccc(S(=O)(=O)O)cc2)cc1)c1ccc(N...,CM,1
2940,NC(=O)CCN(CCC(N)=O)CCC(N)=O,CI,0
2941,Cn1nc2ccccc2cc1=O,CI,0
2942,C[n+]1c(C=NNC(=O)c2ccc(C(=O)NN=Cc3cn4ccccc4[n+...,CI,0


In [19]:
arguments = [
    '--data_path', 'HIV_2.csv',
    '--dataset_type', 'classification',
    '--save_dir', 'test_checkpoints_multimolecule',
    '--epochs', '30',
    '--save_smiles_splits',
    '--quiet',
    '--batch_size', '64',
    '--ignore_columns', 'activity',
    '--depth', '5',
    '--hidden_size', '300'
]

args = chemprop.args.TrainArgs().parse_args(arguments)

In [ ]:
mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)

2943it [00:00, 20570.94it/s]
100%|██████████| 2943/2943 [00:00<00:00, 3194.58it/s]
Fold 0
2943it [00:00, 225499.39it/s]
100%|██████████| 37/37 [00:09<00:00,  6.07it/s]
                                               
100%|██████████| 37/37 [00:04<00:00,  5.95it/s]
                                               
 97%|█████████▋| 36/37 [00:03<00:00,  9.09it/s]
                                               
100%|██████████| 37/37 [00:03<00:00, 10.31it/s]
                                               
 97%|█████████▋| 36/37 [00:04<00:00,  9.22it/s]
                                               
100%|██████████| 37/37 [00:04<00:00,  9.93it/s]
                                               
 97%|█████████▋| 36/37 [00:03<00:00, 10.48it/s]
                                               
100%|██████████| 37/37 [00:04<00:00, 10.73it/s]
                                               
100%|██████████| 37/37 [00:03<00:00,  9.93it/s]
                                               
100%|██████████|

In [22]:
mean_score, std_score

(0.8328736900165471, 0.0)

In [23]:
bp_df = pd.read_csv("BBBP.csv")
bp_df.head()

,num,name,p_np,smiles
0,1,Propanolol,1,[Cl].CC(C)NCC(O)COc1cccc2ccccc12
1,2,Terbutylchlorambucil,1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl
2,3,40730,1,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...
3,4,24,1,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C
4,5,cloxacillin,1,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...


In [24]:
bp_df.tail()

,num,name,p_np,smiles
2045,2049,licostinel,1,C1=C(Cl)C(=C(C2=C1NC(=O)C(N2)=O)[N+](=O)[O-])Cl
2046,2050,ademetionine(adenosyl-methionine),1,[C@H]3([N]2C1=C(C(=NC=N1)N)N=C2)[C@@H]([C@@H](...
2047,2051,mesocarb,1,[O+]1=N[N](C=C1[N-]C(NC2=CC=CC=C2)=O)C(CC3=CC=...
2048,2052,tofisoline,1,C1=C(OC)C(=CC2=C1C(=[N+](C(=C2CC)C)[NH-])C3=CC...
2049,2053,azidamfenicol,1,[N+](=NCC(=O)N[C@@H]([C@H](O)C1=CC=C([N+]([O-]...


In [25]:
bp_df.drop(['num', 'name', 'p_np'], axis=1).to_csv('BBBP_2.csv', index=False)

In [26]:
bp_df_2 = pd.read_csv("BBBP_2.csv")
bp_df_2.head()
bp_df_2.tail()

,smiles
0,[Cl].CC(C)NCC(O)COc1cccc2ccccc12
1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl
2,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...
3,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C
4,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...


,smiles
2045,C1=C(Cl)C(=C(C2=C1NC(=O)C(N2)=O)[N+](=O)[O-])Cl
2046,[C@H]3([N]2C1=C(C(=NC=N1)N)N=C2)[C@@H]([C@@H](...
2047,[O+]1=N[N](C=C1[N-]C(NC2=CC=CC=C2)=O)C(CC3=CC=...
2048,C1=C(OC)C(=CC2=C1C(=[N+](C(=C2CC)C)[NH-])C3=CC...
2049,[N+](=NCC(=O)N[C@@H]([C@H](O)C1=CC=C([N+]([O-]...


In [27]:
arguments = [
    '--test_path', 'BBBP_2.csv',
    '--preds_path', 'BBBP_preds.csv',
    '--checkpoint_dir', 'test_checkpoints_multimolecule'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

Loading training args
Setting molecule featurization parameters to default.
Loading data


2050it [00:00, 230685.03it/s]
100%|██████████| 2050/2050 [00:00<00:00, 144470.78it/s]

Validating SMILES



[19:56:48] Explicit valence for atom # 1 N, 4, is greater than permitted
[19:56:48] WARNING: not removing hydrogen atom without neighbors
[19:56:48] Explicit valence for atom # 6 N, 4, is greater than permitted
[19:56:48] WARNING: not removing hydrogen atom without neighbors
[19:56:48] WARNING: not removing hydrogen atom without neighbors
[19:56:48] WARNING: not removing hydrogen atom without neighbors
[19:56:48] WARNING: not removing hydrogen atom without neighbors
[19:56:48] WARNING: not removing hydrogen atom without neighbors
[19:56:48] WARNING: not removing hydrogen atom without neighbors
[19:56:48] Explicit valence for atom # 6 N, 4, is greater than permitted
[19:56:48] WARNING: not removing hydrogen atom without neighbors
[19:56:48] WARNING: not removing hydrogen atom without neighbors
[19:56:48] WARNING: not removing hydrogen atom without neighbors
[19:56:48] WARNING: not removing hydrogen atom without neighbors
[19:56:48] Explicit valence for atom # 11 N, 4, is greater than p

Test size = 2,039


  0%|          | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".
Moving model to cuda



100%|██████████| 1/1 [00:06<00:00,  6.30s/it]


Saving predictions to BBBP_preds.csv
Elapsed time = 0:00:07


In [28]:
bp_preds_df = pd.read_csv("BBBP_preds.csv")
bp_preds_df.head()

,smiles,HIV_active
0,[Cl].CC(C)NCC(O)COc1cccc2ccccc12,0.07791923731565475
1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl,0.052813779562711716
2,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...,0.5678612589836121
3,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C,0.0569111704826355
4,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...,0.4187469184398651


In [29]:
bp_preds_df.tail()

,smiles,HIV_active
2045,C1=C(Cl)C(=C(C2=C1NC(=O)C(N2)=O)[N+](=O)[O-])Cl,0.1297084391117096
2046,[C@H]3([N]2C1=C(C(=NC=N1)N)N=C2)[C@@H]([C@@H](...,0.23322035372257233
2047,[O+]1=N[N](C=C1[N-]C(NC2=CC=CC=C2)=O)C(CC3=CC=...,0.3322729766368866
2048,C1=C(OC)C(=CC2=C1C(=[N+](C(=C2CC)C)[NH-])C3=CC...,0.3129490911960602
2049,[N+](=NCC(=O)N[C@@H]([C@H](O)C1=CC=C([N+]([O-]...,0.2837357521057129


In [30]:
bp_preds_df.describe()

,smiles,HIV_active
count,2050,2050
unique,2050,2004
top,[N+](=NCC(=O)N[C@@H]([C@H](O)C1=CC=C([N+]([O-]...,Invalid SMILES
freq,1,11


In [31]:
bp_preds_df = bp_preds_df[bp_preds_df['HIV_active'] != "Invalid SMILES"]
bp_preds_df.describe()

,smiles,HIV_active
count,2039,2039
unique,2039,2003
top,[N+](=NCC(=O)N[C@@H]([C@H](O)C1=CC=C([N+]([O-]...,0.0426582507789135
freq,1,3


In [49]:
bp_preds_df['HIV_active'] = bp_preds_df['HIV_active'].astype(float)

In [50]:
bp_preds_df['HIV_active_2'] = bp_preds_df['HIV_active'].apply(lambda x: 1 if x > 0.8 else 0)
bp_preds_df.head()


,smiles,HIV_active,HIV_active_2
0,[Cl].CC(C)NCC(O)COc1cccc2ccccc12,0.077919,0
1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl,0.052814,0
2,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...,0.567861,0
3,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C,0.056911,0
4,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...,0.418747,0


In [51]:
bp_preds_df.describe()

,HIV_active,HIV_active_2
count,2039.000000,2039.000000
mean,0.263883,0.009809
std,0.201033,0.098576
min,0.000762,0.000000
25%,0.097246,0.000000
50%,0.208566,0.000000
75%,0.408756,0.000000
max,0.988302,1.000000


In [52]:
# Filter rows where 'target_column' is equal to 1
bp_preds_df_filtered = bp_preds_df[bp_preds_df['HIV_active_2'] == 1]
bp_preds_df_filtered


,smiles,HIV_active,HIV_active_2
11,CC1=CN([C@H]2C[C@H](F)[C@@H](CO)O2)C(=O)NC1=O,0.944161,1
235,OC[C@H]1O[C@H](C[C@@H]1O)N2C=C(F)C(=O)NC2=O,0.864316,1
270,Cc1nnc2CN=C(c3ccccc3)c4cc(Cl)ccc4n12,0.802165,1
279,OC[C@H]1O[C@@H](OC2=C(Oc3cc(O)cc(O)c3C2=O)c4cc...,0.880075,1
289,OC[C@@H]1CC[C@@H](O1)n2cnc3C(=O)N=CNc23,0.812233,1
319,CC1=CN([C@@H]2O[C@H](CO)C=C2)C(=O)NC1=O,0.930296,1
445,CC1=CN([C@H]2C[C@H](N=[N+]=[N-])[C@@H](CO)O2)C...,0.985429,1
540,Cc1nnc2CN=C(c3ccccc3Cl)c4cc(Cl)ccc4n12,0.856978,1
581,CN1CCC23C4CCC(O)C2Oc5c(O)ccc(CC14)c35,0.816521,1
711,Cc1ncc2CN=C(c3ccccc3F)c4cc(Cl)ccc4n12.OC(=O)\C...,0.842499,1


In [53]:
smiles_to_check = bp_preds_df_filtered['smiles'].to_list()

In [54]:
hiv_df_sampled_2[hiv_df_sampled_2['smiles'].isin(smiles_to_check)]

,smiles,activity,HIV_active


In [55]:
hiv_df[hiv_df['smiles'].isin(smiles_to_check)]

,smiles,activity,HIV_active


In [56]:
bp_df[bp_df['smiles'].isin(smiles_to_check)]

,num,name,p_np,smiles
11,12,alovudine,1,CC1=CN([C@H]2C[C@H](F)[C@@H](CO)O2)C(=O)NC1=O
235,237,floxuridine,0,OC[C@H]1O[C@H](C[C@@H]1O)N2C=C(F)C(=O)NC2=O
270,272,Alprazolam,1,Cc1nnc2CN=C(c3ccccc3)c4cc(Cl)ccc4n12
279,281,Isoquercitrin,0,OC[C@H]1O[C@@H](OC2=C(Oc3cc(O)cc(O)c3C2=O)c4cc...
289,291,Didanosine,0,OC[C@@H]1CC[C@@H](O1)n2cnc3C(=O)N=CNc23
319,321,Stavudine,1,CC1=CN([C@@H]2O[C@H](CO)C=C2)C(=O)NC1=O
445,447,zidovudine,0,CC1=CN([C@H]2C[C@H](N=[N+]=[N-])[C@@H](CO)O2)C...
540,542,Triazolam,1,Cc1nnc2CN=C(c3ccccc3Cl)c4cc(Cl)ccc4n12
581,583,dihydromorphine,1,CN1CCC23C4CCC(O)C2Oc5c(O)ccc(CC14)c35
711,713,midazolam maleate,1,Cc1ncc2CN=C(c3ccccc3F)c4cc(Cl)ccc4n12.OC(=O)\C...


In [57]:
bp_df_final = pd.merge(bp_df[bp_df['smiles'].isin(smiles_to_check)], bp_preds_df_filtered, on='smiles' )
bp_df_final

,num,name,p_np,smiles,HIV_active,HIV_active_2
0,12,alovudine,1,CC1=CN([C@H]2C[C@H](F)[C@@H](CO)O2)C(=O)NC1=O,0.944161,1
1,237,floxuridine,0,OC[C@H]1O[C@H](C[C@@H]1O)N2C=C(F)C(=O)NC2=O,0.864316,1
2,272,Alprazolam,1,Cc1nnc2CN=C(c3ccccc3)c4cc(Cl)ccc4n12,0.802165,1
3,281,Isoquercitrin,0,OC[C@H]1O[C@@H](OC2=C(Oc3cc(O)cc(O)c3C2=O)c4cc...,0.880075,1
4,291,Didanosine,0,OC[C@@H]1CC[C@@H](O1)n2cnc3C(=O)N=CNc23,0.812233,1
5,321,Stavudine,1,CC1=CN([C@@H]2O[C@H](CO)C=C2)C(=O)NC1=O,0.930296,1
6,447,zidovudine,0,CC1=CN([C@H]2C[C@H](N=[N+]=[N-])[C@@H](CO)O2)C...,0.985429,1
7,542,Triazolam,1,Cc1nnc2CN=C(c3ccccc3Cl)c4cc(Cl)ccc4n12,0.856978,1
8,583,dihydromorphine,1,CN1CCC23C4CCC(O)C2Oc5c(O)ccc(CC14)c35,0.816521,1
9,713,midazolam maleate,1,Cc1ncc2CN=C(c3ccccc3F)c4cc(Cl)ccc4n12.OC(=O)\C...,0.842499,1


In [58]:
bp_df_final.to_csv('HIV_result.csv', index=False)

In [59]:
sub_df = pd.read_csv("substances.csv")
sub_df.head()

,zinc_id,smiles
0,ZINC000000000027,N[C@@H](CCc1ccc(N(CCCl)CCCl)cc1)C(=O)O
1,ZINC000016090786,N[C@H](CCc1ccc(N(CCCl)CCCl)cc1)C(=O)O
2,ZINC000001763088,N[C@H](CCCc1ccc(N(CCCl)CCCl)cc1)C(=O)O
3,ZINC000002033385,N[C@@H](CCCc1ccc(N(CCCl)CCCl)cc1)C(=O)O
4,ZINC000000001673,N[C@@H](Cc1ccc(N(CCCl)CCCl)cc1)C(=O)O


In [60]:
sub_df.tail()

,zinc_id,smiles
46,ZINC000196349655,O=C(O)CCSc1ccc(N(CCCl)CCCl)cc1
47,ZINC000064454242,N=NCCCc1ccc(N(CCCl)CCCl)cc1
48,ZINC000005161807,O=C(O)C/C=C/c1ccc(N(CCCl)CCCl)cc1
49,ZINC000001682294,O=C(O)CCOc1ccc(N(CCCl)CCCl)cc1
50,ZINC000079564304,O=C(O)CNC(=O)c1ccc(N(CCCl)CCCl)cc1


In [61]:
sub_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   zinc_id  51 non-null     object
 1   smiles   51 non-null     object
dtypes: object(2)
memory usage: 944.0+ bytes


In [62]:
arguments = [
    '--test_path', 'substances.csv',
    '--preds_path', 'substances_preds.csv',
    '--checkpoint_dir', 'test_checkpoints_multimolecule',
    '--smiles_columns', 'smiles'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

Loading training args
Setting molecule featurization parameters to default.
Loading data


51it [00:00, 60426.41it/s]
100%|██████████| 51/51 [00:00<00:00, 76780.15it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Validating SMILES
Test size = 51


  0%|          | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".
Moving model to cuda



100%|██████████| 1/1 [00:01<00:00,  1.20s/it]

Saving predictions to substances_preds.csv
Elapsed time = 0:00:01


In [63]:
fda_df = pd.read_csv("fda_approved.csv")
fda_df.head()

,zinc_id,smiles
0,ZINC000001530427,C[C@@H]1O[C@@H]1P(=O)(O)O
1,ZINC000003807804,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1
2,ZINC000000120286,Nc1nc(N)c2nc(-c3ccccc3)c(N)nc2n1
3,ZINC000242548690,C[C@H]1O[C@@H](O[C@H]2[C@@H](O)C[C@H](O[C@H]3[...
4,ZINC000000008492,Oc1cccc2cccnc12


In [64]:
arguments = [
    '--test_path', 'fda_approved.csv',
    '--preds_path', 'fda_approved_preds.csv',
    '--checkpoint_dir', 'test_checkpoints_multimolecule',
    '--smiles_columns', 'smiles'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

Loading training args
Setting molecule featurization parameters to default.
Loading data


892it [00:00, 90156.61it/s]
100%|██████████| 892/892 [00:00<00:00, 51982.98it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Validating SMILES
Test size = 892


  0%|          | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".
Moving model to cuda



100%|██████████| 1/1 [00:03<00:00,  3.84s/it]

Saving predictions to fda_approved_preds.csv
Elapsed time = 0:00:04


In [65]:
fda_preds_df = pd.read_csv("fda_approved_preds.csv")
fda_preds_df.head()

,zinc_id,smiles,HIV_active
0,ZINC000001530427,C[C@@H]1O[C@@H]1P(=O)(O)O,0.085422
1,ZINC000003807804,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,0.303771
2,ZINC000000120286,Nc1nc(N)c2nc(-c3ccccc3)c(N)nc2n1,0.093442
3,ZINC000242548690,C[C@H]1O[C@@H](O[C@H]2[C@@H](O)C[C@H](O[C@H]3[...,0.565260
4,ZINC000000008492,Oc1cccc2cccnc12,0.089892


In [66]:
fda_preds_df = fda_preds_df[fda_preds_df['HIV_active'] != "Invalid SMILES"]
fda_preds_df.describe()
fda_preds_df['HIV_active'] = fda_preds_df['HIV_active'].astype(float)
fda_preds_df['HIV_active_2'] = fda_preds_df['HIV_active'].apply(lambda x: 1 if x > 0.8 else 0)
fda_preds_df.head()

,HIV_active
count,892.000000
mean,0.257912
std,0.204399
min,0.003855
25%,0.097084
50%,0.190803
75%,0.389287
max,0.992705


,zinc_id,smiles,HIV_active,HIV_active_2
0,ZINC000001530427,C[C@@H]1O[C@@H]1P(=O)(O)O,0.085422,0
1,ZINC000003807804,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,0.303771,0
2,ZINC000000120286,Nc1nc(N)c2nc(-c3ccccc3)c(N)nc2n1,0.093442,0
3,ZINC000242548690,C[C@H]1O[C@@H](O[C@H]2[C@@H](O)C[C@H](O[C@H]3[...,0.565260,0
4,ZINC000000008492,Oc1cccc2cccnc12,0.089892,0


In [67]:
# Filter rows where 'target_column' is equal to 1
fda_preds_df_filtered = fda_preds_df[fda_preds_df['HIV_active_2'] == 1]
fda_preds_df_filtered

,zinc_id,smiles,HIV_active,HIV_active_2
31,ZINC000003816287,CNC(=O)c1ccccc1Sc1ccc2c(/C=C/c3ccccn3)n[nH]c2c1,0.819102,1
47,ZINC000003813010,O=c1[nH]c(=O)n([C@H]2C[C@H](O)[C@@H](CO)O2)cc1F,0.884875,1
81,ZINC000003818726,O=C(/C=C/c1cccc(S(=O)(=O)Nc2ccccc2)c1)NO,0.804625,1
94,ZINC000068153186,CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2...,0.938759,1
197,ZINC000000005423,Cc1nc(-c2ccc(OCC(C)C)c(C#N)c2)sc1C(=O)O,0.854534,1
276,ZINC000000002212,Cc1nnc2n1-c1ccc(Cl)cc1C(c1ccccc1Cl)=NC2,0.856978,1
321,ZINC000013597823,O=c1[nH]cnc2c1ncn2[C@H]1CC[C@@H](CO)O1,0.851600,1
340,ZINC000001530621,CCN[C@H]1C[C@H](C)S(=O)(=O)c2sc(S(N)(=O)=O)cc21,0.824345,1
499,ZINC000000896717,COc1cc(/C(O)=N/S(=O)(=O)c2ccccc2C)ccc1Cc1cn(C)...,0.809218,1
540,ZINC000004474564,CC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCC(=O)O,0.824716,1


In [68]:
smiles_to_check = fda_preds_df_filtered['smiles'].to_list()
print(f"smiles to check: {smiles_to_check}")

smiles to check: ['CNC(=O)c1ccccc1Sc1ccc2c(/C=C/c3ccccn3)n[nH]c2c1', 'O=c1[nH]c(=O)n([C@H]2C[C@H](O)[C@@H](CO)O2)cc1F', 'O=C(/C=C/c1cccc(S(=O)(=O)Nc2ccccc2)c1)NO', 'CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2F)c(-c2ccnc(N)n2)s1', 'Cc1nc(-c2ccc(OCC(C)C)c(C#N)c2)sc1C(=O)O', 'Cc1nnc2n1-c1ccc(Cl)cc1C(c1ccccc1Cl)=NC2', 'O=c1[nH]cnc2c1ncn2[C@H]1CC[C@@H](CO)O1', 'CCN[C@H]1C[C@H](C)S(=O)(=O)c2sc(S(N)(=O)=O)cc21', 'COc1cc(/C(O)=N/S(=O)(=O)c2ccccc2C)ccc1Cc1cn(C)c2ccc(NC(=O)OC3CCCC3)cc12', 'CC/C=C\\C/C=C\\C/C=C\\C/C=C\\C/C=C\\C/C=C\\CCC(=O)O', 'COc1ccc(-c2cc(=O)c3c(O)cc(O)cc3o2)cc1O', 'N[C@@H](Cc1cc(I)c(Oc2cc(I)c(O)c(I)c2)c(I)c1)C(=O)O', 'Cc1cc(-c2ccc(/N=N/c3c(S(=O)(=O)O)cc4cc(S(=O)(=O)O)cc(N)c4c3O)c(C)c2)ccc1/N=N/c1c(S(=O)(=O)O)cc2cc(S(=O)(=O)O)cc(N)c2c1O', 'C[C@H]1CNc2c(cccc2S(=O)(=O)N[C@@H](CCCNC(=N)N)C(=O)N2CC[C@@H](C)C[C@@H]2C(=O)O)C1', 'Cc1nnc2n1-c1ccc(Cl)cc1C(c1ccccc1)=NC2', 'Cc1cn([C@H]2C=C[C@@H](CO)O2)c(=O)[nH]c1=O', 'CC1(C)[C@H](C(=O)O)N2C(=O)C[C@H]2S1(=O)=O', 'Cc1cn([C@H]2C[C@H](N=[

In [69]:
hiv_df_sampled_2[hiv_df_sampled_2['smiles'].isin(smiles_to_check)]

,smiles,activity,HIV_active


In [70]:
hiv_df[hiv_df['smiles'].isin(smiles_to_check)]

,smiles,activity,HIV_active


In [71]:
bp_df[bp_df['smiles'].isin(smiles_to_check)]

,num,name,p_np,smiles


In [72]:
fda_df[fda_df['smiles'].isin(smiles_to_check)]

,zinc_id,smiles
31,ZINC000003816287,CNC(=O)c1ccccc1Sc1ccc2c(/C=C/c3ccccn3)n[nH]c2c1
47,ZINC000003813010,O=c1[nH]c(=O)n([C@H]2C[C@H](O)[C@@H](CO)O2)cc1F
81,ZINC000003818726,O=C(/C=C/c1cccc(S(=O)(=O)Nc2ccccc2)c1)NO
94,ZINC000068153186,CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2...
197,ZINC000000005423,Cc1nc(-c2ccc(OCC(C)C)c(C#N)c2)sc1C(=O)O
276,ZINC000000002212,Cc1nnc2n1-c1ccc(Cl)cc1C(c1ccccc1Cl)=NC2
321,ZINC000013597823,O=c1[nH]cnc2c1ncn2[C@H]1CC[C@@H](CO)O1
340,ZINC000001530621,CCN[C@H]1C[C@H](C)S(=O)(=O)c2sc(S(N)(=O)=O)cc21
499,ZINC000000896717,COc1cc(/C(O)=N/S(=O)(=O)c2ccccc2C)ccc1Cc1cn(C)...
540,ZINC000004474564,CC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCC(=O)O


In [73]:
fda_df_final = pd.merge(fda_df[fda_df['smiles'].isin(smiles_to_check)], fda_preds_df_filtered, on='smiles' )
fda_df_final

,zinc_id_x,smiles,zinc_id_y,HIV_active,HIV_active_2
0,ZINC000003816287,CNC(=O)c1ccccc1Sc1ccc2c(/C=C/c3ccccn3)n[nH]c2c1,ZINC000003816287,0.819102,1
1,ZINC000003813010,O=c1[nH]c(=O)n([C@H]2C[C@H](O)[C@@H](CO)O2)cc1F,ZINC000003813010,0.884875,1
2,ZINC000003818726,O=C(/C=C/c1cccc(S(=O)(=O)Nc2ccccc2)c1)NO,ZINC000003818726,0.804625,1
3,ZINC000068153186,CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2...,ZINC000068153186,0.938759,1
4,ZINC000000005423,Cc1nc(-c2ccc(OCC(C)C)c(C#N)c2)sc1C(=O)O,ZINC000000005423,0.854534,1
5,ZINC000000002212,Cc1nnc2n1-c1ccc(Cl)cc1C(c1ccccc1Cl)=NC2,ZINC000000002212,0.856978,1
6,ZINC000013597823,O=c1[nH]cnc2c1ncn2[C@H]1CC[C@@H](CO)O1,ZINC000013597823,0.851600,1
7,ZINC000001530621,CCN[C@H]1C[C@H](C)S(=O)(=O)c2sc(S(N)(=O)=O)cc21,ZINC000001530621,0.824345,1
8,ZINC000000896717,COc1cc(/C(O)=N/S(=O)(=O)c2ccccc2C)ccc1Cc1cn(C)...,ZINC000000896717,0.809218,1
9,ZINC000004474564,CC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCC(=O)O,ZINC000004474564,0.824716,1


In [74]:
fda_df_final.to_csv('fda_approved_result.csv', index=False)

In [121]:
# !wget https://zinc15.docking.org/substances/subsets/named.csv

--2024-03-10 05:58:28--  https://zinc15.docking.org/substances/subsets/named.csv
Resolving zinc15.docking.org (zinc15.docking.org)... 169.230.75.4
Connecting to zinc15.docking.org (zinc15.docking.org)|169.230.75.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘named.csv.1’

named.csv.1             [ <=>                ]   9.28K  --.-KB/s    in 0.04s   

2024-03-10 05:58:29 (242 KB/s) - ‘named.csv.1’ saved [9499]



In [75]:
zinc_df = pd.read_csv("named.csv")
zinc_df.head()
zinc_df.tail()

,zinc_id,smiles
0,ZINC000030727788,C=C[C@]1(C)C[C@@H](OC(=O)CSC(C)(C)CNC(=O)[C@H]...
1,ZINC000150377216,CCCCCC/C=C\C/C=C\CCCCCCCC(=O)OC[C@H](COCCCCCCC...
2,ZINC000100780125,CC(=O)O[C@H]1C[C@](C)(O)[C@@H]2CC=C(C)[C@@H]2[...
3,ZINC000006580536,O=C(O)[C@H](Cc1ccccc1)N(CCCl)CCCl
4,ZINC000150351802,O=C1C[C@H](c2ccc(O)c(O)c2)Oc2c1c(O)cc(O[C@H]1O...


,zinc_id,smiles
34595,ZINC000005999135,COc1cc([C@@H]2Oc3c(OC)cc(/C=C/CO)cc3[C@H]2CO)c...
34596,ZINC000084710404,COC(=O)c1cc(OC)c2cc(OC)c(OC)c(O)c2c1O
34597,ZINC000150369761,CC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCC(=O)O[...
34598,ZINC000095098911,CCCCCCCCCCCC[C@H](O)[C@H]1CC[C@H]([C@H](O)CCCC...
34599,ZINC000000001009,C[N+](C)([O-])CC/C=C1\c2ccccc2C=Cc2c(Cl)cccc21


In [76]:
arguments = [
    '--test_path', 'named.csv',
    '--preds_path', 'named_preds.csv',
    '--checkpoint_dir', 'test_checkpoints_multimolecule',
    '--smiles_columns', 'smiles'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

Loading training args
Setting molecule featurization parameters to default.
Loading data


34600it [00:00, 131342.23it/s]
100%|██████████| 34600/34600 [00:00<00:00, 43207.69it/s]


Validating SMILES


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Test size = 34,600


  0%|          | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".
Moving model to cuda



100%|██████████| 1/1 [02:38<00:00, 158.26s/it]


Saving predictions to named_preds.csv
Elapsed time = 0:02:52


In [78]:
zinc_preds_df = pd.read_csv("named_preds.csv")
zinc_preds_df.head()
zinc_preds_df = zinc_preds_df[zinc_preds_df['HIV_active'] != "Invalid SMILES"]
zinc_preds_df.describe()
zinc_preds_df['HIV_active'] = zinc_preds_df['HIV_active'].astype(float)
zinc_preds_df['HIV_active_2'] = zinc_preds_df['HIV_active'].apply(lambda x: 1 if x > 0.8 else 0)
zinc_preds_df.head()

,zinc_id,smiles,HIV_active
0,ZINC000030727788,C=C[C@]1(C)C[C@@H](OC(=O)CSC(C)(C)CNC(=O)[C@H]...,0.345203
1,ZINC000150377216,CCCCCC/C=C\C/C=C\CCCCCCCC(=O)OC[C@H](COCCCCCCC...,0.619356
2,ZINC000100780125,CC(=O)O[C@H]1C[C@](C)(O)[C@@H]2CC=C(C)[C@@H]2[...,0.260698
3,ZINC000006580536,O=C(O)[C@H](Cc1ccccc1)N(CCCl)CCCl,0.043098
4,ZINC000150351802,O=C1C[C@H](c2ccc(O)c(O)c2)Oc2c1c(O)cc(O[C@H]1O...,0.716069


,HIV_active
count,34600.000000
mean,0.466442
std,0.243481
min,0.000041
25%,0.251037
50%,0.499464
75%,0.673119
max,0.998601


,zinc_id,smiles,HIV_active,HIV_active_2
0,ZINC000030727788,C=C[C@]1(C)C[C@@H](OC(=O)CSC(C)(C)CNC(=O)[C@H]...,0.345203,0
1,ZINC000150377216,CCCCCC/C=C\C/C=C\CCCCCCCC(=O)OC[C@H](COCCCCCCC...,0.619356,0
2,ZINC000100780125,CC(=O)O[C@H]1C[C@](C)(O)[C@@H]2CC=C(C)[C@@H]2[...,0.260698,0
3,ZINC000006580536,O=C(O)[C@H](Cc1ccccc1)N(CCCl)CCCl,0.043098,0
4,ZINC000150351802,O=C1C[C@H](c2ccc(O)c(O)c2)Oc2c1c(O)cc(O[C@H]1O...,0.716069,0


In [79]:
# Filter rows where 'target_column' is equal to 1
zinc_preds_df_filtered = zinc_preds_df[zinc_preds_df['HIV_active_2'] == 1]
zinc_preds_df_filtered

,zinc_id,smiles,HIV_active,HIV_active_2
21,ZINC000040753343,COc1cc(/C=C/c2cc(O)c(CC=C(C)C)c(O)c2)cc2c1O[C@...,0.934274,1
37,ZINC000014615844,CC1(C)OC(=O)C=C[C@@]2(C)[C@@H]1CC(=O)[C@@]1(C)...,0.845315,1
91,ZINC000150366864,COc1cc(-c2[o+]c3cc(O)cc(O)c3cc2O[C@H]2O[C@H](C...,0.843619,1
110,ZINC000049888739,CCCCCC(=O)c1c(O)c2c(c3c1O[C@@]1(O)[C@H]([C@@H]...,0.885498,1
124,ZINC000150343906,CCCCC/C=C\C/C=C\CCCCCCCC(=O)OC[C@H](CO[P@@](=O...,0.822896,1
...,...,...,...,...
34571,ZINC000013414448,COc1cc(O)c2c3c1OC(C)(C)[C@H]3Cc1c-2oc2c3c(cc(O...,0.907510,1
34573,ZINC000014647291,COc1cc(-c2oc3cc(O[C@@H]4O[C@H](CO)[C@@H](O)[C@...,0.804389,1
34585,ZINC000000538635,CC(C)c1nc(COC(N)=O)n(Cc2ccncc2)c1Sc1cc(Cl)cc(C...,0.801660,1
34590,ZINC000049767976,COC(=O)c1cc(OC)cc(O)c1C(=O)c1c(O)cc(C)c(C2=CC(...,0.924955,1


In [80]:
zinc_preds_df_filtered.to_csv('zinc_final_result.csv', index=False)

In [81]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [82]:
!mkdir '/content/drive/My Drive/Chemprop_Backup_HIV_2/'

In [83]:
!pwd

/content


In [84]:
!ls -al

total 9992
drwxr-xr-x 1 root root    4096 Mar 10 20:28 .
drwxr-xr-x 1 root root    4096 Mar 10 19:43 ..
-rw-r--r-- 1 root root  107579 Mar 10 19:56 BBBP_2.csv
-rw-r--r-- 1 root root  148743 Mar 10 19:50 BBBP.csv
-rw-r--r-- 1 root root  149793 Mar 10 19:56 BBBP_preds.csv
drwxr-xr-x 4 root root    4096 Mar  7 14:31 .config
drwx------ 6 root root    4096 Mar 10 20:28 drive
-rw-r--r-- 1 root root   61865 Mar 10 20:00 fda_approved.csv
-rw-r--r-- 1 root root   79358 Mar 10 20:16 fda_approved_preds.csv
-rw-r--r-- 1 root root    1998 Mar 10 20:21 fda_approved_result.csv
-rw-r--r-- 1 root root  174576 Mar 10 19:51 HIV_2.csv
-rw-r--r-- 1 root root 2193844 Mar 10 19:50 HIV.csv
-rw-r--r-- 1 root root    1887 Mar 10 20:16 HIV_result.csv
-rw-r--r-- 1 root root 3160219 Mar 10 20:22 named.csv
-rw-r--r-- 1 root root 3826178 Mar 10 20:25 named_preds.csv
drwxr-xr-x 1 root root    4096 Mar  7 14:32 sample_data
-rw-r--r-- 1 root root    2767 Mar 10 20:00 substances.csv
-rw-r--r-- 1 root root    3805 Mar 10

In [85]:
!cp -r /content/*.csv "/content/drive/My Drive/Chemprop_Backup_HIV_2/"

In [86]:
!cp -r /content/test_checkpoints_multimolecule/ "/content/drive/My Drive/Chemprop_Backup_HIV_2/"